## 分布式训练

分布式训练有两种实现模式：模型并行和数据并行
- 模型并行是将一个网络模型拆分成多份，拆分后的模型分到多个设备上训练，每个设备的训练数据是相同的。模型并行可以节省内存，但是应用较为受限，一般适用于如下两个场景：
    - 模型架构过大：完整的模型无法放入单个GPU。
    - 模型的结构设计相对独立：当网络模型的设计结构可以并行化时，采用模型并行的方式。
- 数据并行时每次读取多份数据，然后输入到多个设备上的模型。每个设备上的模型是完全相同的，但是输入的数据不同，因此每个设备的模型计算出的梯度是不同的。如果每个设备的梯度只更新当前设备的模型，就会导致下次训练时，每个模型的参数都不相同，因此需要梯度同步的机制，有两种方式：
    - PRC通信方式
    - NCCL2通信方式

## 训练过程优化

1. 计算分类准确率，观察模型训练效果。
    - 交叉熵损失函数只能作为优化目标，无法直接准确衡量模型的训练效果。准确率可以直接衡量训练效果，但由于其离散性质，不适合做为损失函数优化神经网络。
2. 检查模型训练过程，识别潜在问题。
    - 如果模型的损失或者评估指标表现异常，通常需要打印模型每一层的输入和输出来定位问题，分析每一层的内容来获取错误的原因。
3. 加入校验和测试，更好评价模型效果。
    - 理想的模型训练结果是在训练集和验证集上均有较高的准确率，如果训练集的准确率低于验证集，说明网络训练程度不够；如果训练集的准确率高于验证集，可能是发生了过拟合现象。通过在优化目标中加入正则化项的办法，解决过拟合的问题。
4. 加入正则化项，避免模型过拟合。
    - 飞桨框架支持为整体参数加入正则化项，这是通常的做法。此外，飞桨框架也支持为某一层或某一部分的网络单独加入正则化项，以达到精细调整参数训练的效果。
5. 可视化分析
    - 用户不仅可以通过打印或使用matplotlib库作图，飞桨还提供了更专业的可视化分析工具VisualDL，提供便捷的可视化分析方法。

In [13]:
import numpy as np
import matplotlib.pyplot as plt

import paddle
from paddle.nn import Conv2D, MaxPool2D, Linear
import paddle.nn.functional as F

from PIL import Image
import os
import random
import gzip
import json

In [14]:
# 构建一个加载数据的迭代器
class MnistDataset(paddle.io.Dataset):
    def __init__(self, mode):
        # 读取原始数据
        datafile = '../datasets/mnist.json.gz'
        data = json.load(gzip.open(datafile))
        
        # 划分训练集、验证集和测试集
        train_set, valid_set, test_set = data
        
        # 数据集相关参数：图片宽、高度
        self.IMG_ROWS = 28
        self.IMG_COLS = 28
        
        # 拆分数据和标签
        if mode=='train':
            imgs, labels = train_set[0], train_set[1]
        elif mode=='valid':
            imgs, labels = valid_set[0], valid_set[1]
        elif mode=='test':
            imgs, labels = test_set[0], test_set[1]
        else:
            raise Exception("Mode can only be one of ['train', 'valid', 'test']")
                            
        # 校验数据
        imgs_length = len(imgs)
        assert len(imgs) == len(labels), \
            'Length of imgs({}) should be the same as labels({})'.format(len(imgs), len(labels))
                            
        self.imgs = imgs
        self.labels = labels
    
    def __getitem__(self, idx):
        img = np.reshape(self.imgs[idx], [1, self.IMG_ROWS, self.IMG_COLS]).astype('float32')
        label = np.reshape(self.labels[idx], [1]).astype('int64')
        
        return img, label
    
    def __len__(self):
        return len(self.imgs)

In [15]:
# 使用训练模式，迭代器每次迭代只返回batch=1的数据
train_dataset = MnistDataset(mode='train')

# 使用paddle.io.DataLoader，返回的是一个批次数据迭代器，并且是异步的
train_loader = paddle.io.DataLoader(train_dataset, batch_size=100, shuffle=True, drop_last=True)

# 加载验证数据集
valid_dataset = MnistDataset(mode='valid')
valid_loader = paddle.io.DataLoader(valid_dataset, batch_size=128, drop_last=True)

In [22]:
# 定义多层卷积神经网络
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()

        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输出维度是10
        self.fc = Linear(in_features=980, out_features=10)
        
    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    # 卷积层激活函数使用Relu，全连接层激活函数使用softmax
    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], 980])
        x = self.fc(x)
        
        # 计算准确率
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label=label)
            return x, acc
        else:
            return x

In [27]:
def train(model):
    # 单GPU训练
    use_gpu = False
    # 参数device(str)：此参数确定特定的运行设备，可以使cpu, gpu:x, xpu:x，x是GPU或XPU的编号
    paddle.device.set_device('gpu') if use_gpu else paddle.device.set_device('cpu')
    
    # 开启训练模式
    model.train()
    
    # 加入正则化项，避免模型过拟合，并可以调节其权重
    opt = paddle.optimizer.Adam(learning_rate=0.001, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5), parameters=model.parameters())
    
    EPOCH_NUM = 5
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            # 准备数据
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)
            
            # 前向计算的过程
            predicts, acc = model(images, labels)
            
            # 计算损失，使用交叉熵损失函数，取一个批次样本损失的平均值
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            
            # 每训练了200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
            
            # 后向传播，更新参数的过程
            avg_loss.backward()
            # 最小化loss,更新参数
            opt.step()
            # 清除梯度
            opt.clear_grad()
        
    #保存模型参数
    paddle.save(model.state_dict(), '../models/mnist.pdparams')

In [28]:
# Init Model
model = MNIST()

# Start training
train(model)

epoch: 0, batch: 0, loss is: [3.3794203], acc is [0.09]
epoch: 0, batch: 200, loss is: [0.17780504], acc is [0.93]
epoch: 0, batch: 400, loss is: [0.12832578], acc is [0.97]
epoch: 1, batch: 0, loss is: [0.11423808], acc is [0.97]
epoch: 1, batch: 200, loss is: [0.03655875], acc is [0.99]
epoch: 1, batch: 400, loss is: [0.04313245], acc is [0.98]
epoch: 2, batch: 0, loss is: [0.10315643], acc is [0.98]
epoch: 2, batch: 200, loss is: [0.04177022], acc is [0.98]
epoch: 2, batch: 400, loss is: [0.00749094], acc is [1.]
epoch: 3, batch: 0, loss is: [0.0125254], acc is [1.]
epoch: 3, batch: 200, loss is: [0.00650845], acc is [1.]
epoch: 3, batch: 400, loss is: [0.03692623], acc is [0.99]
epoch: 4, batch: 0, loss is: [0.01093927], acc is [1.]
epoch: 4, batch: 200, loss is: [0.04604139], acc is [0.98]
epoch: 4, batch: 400, loss is: [0.04871863], acc is [0.97]
